In [1]:
#Update config.ymal
import os

In [2]:
%pwd

'e:\\Datasets\\Hafeez_DataSets\\chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [6]:
%pwd

'e:\\Datasets\\Hafeez_DataSets\\chicken-Disease-Classification'

In [8]:
from dataclasses import dataclass
from pathlib import Path
import yaml
from cnnClassifier.constants import *
import cnnClassifier.constants
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

def read_yaml(path_to_yaml: Path) -> dict:
    with open(path_to_yaml) as yaml_file:
        content = yaml.safe_load(yaml_file)
    return content

def create_directories(paths: list):
    for path in paths:
        print(f"Creating directory: {path}")  # Debugging output
        os.makedirs(path, exist_ok=True)


class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config['artifacts_root'])])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']
        create_directories([Path(config['root_dir'])])
        return DataIngestionConfig(
            root_dir=Path(config['root_dir']),
            source_URL=config['source_URL'],
            local_data_file=Path(config['local_data_file']),
            unzip_dir=Path(config['unzip_dir'])
        )

import urllib.request as request
import zipfile

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not self.config.local_data_file.exists():
            print("check this",self.config.source_URL)
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            print(f"{filename} downloaded with following info: \n{headers}")
        else:
            print(f"File already exists of size: {os.path.getsize(self.config.local_data_file)} bytes")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        print(unzip_path)
        os.makedirs(unzip_path, exist_ok=True)
        print(self.config.local_data_file)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        print(f"Extracted to {unzip_path}")

# Pipeline creation
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    print(f"An error occurred: {e}")


from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    

# Configuration Manager

from cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:

    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config['artifacts_root']])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config['prepare_base_model']
        create_directories([config['root_dir']])

        return PrepareBaseModelConfig(
            root_dir=Path(config['root_dir']),
            base_model_path=Path(config['base_model_path']),
            updated_base_model_path=Path(config['updated_base_model_path']),
            params_image_size=self.params['IMAGE_SIZE'],
            params_learning_rate=self.params['LEARNING_RATE'],
            params_include_top=self.params['INCLUDE_TOP'],
            params_weights=self.params['WEIGHTS'],
            params_classes=self.params['CLASSES']
        )
# Usage example:
config_manager = ConfigurationManager()
prepare_base_model_config = config_manager.get_prepare_base_model_config()
print(prepare_base_model_config)

Creating directory: artifacts
Creating directory: artifacts\data_ingestion
File already exists of size: 26061716 bytes
artifacts\data_ingestion
artifacts\data_ingestion\data.zip
Extracted to artifacts\data_ingestion
[2024-05-24 19:35:07,851: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-24 19:35:07,851: INFO: common: yaml file: config\params.yaml loaded successfully]
[2024-05-24 19:35:07,851: INFO: common: created directories at: artifacts]
[2024-05-24 19:35:07,851: INFO: common: created directories at: artifacts/prepare_base_model]
PrepareBaseModelConfig(root_dir=WindowsPath('artifacts/prepare_base_model'), base_model_path=WindowsPath('artifacts/prepare_base_model/base_model.h5'), updated_base_model_path=WindowsPath('artifacts/prepare_base_model/base_model_updated.h5'), params_image_size=BoxList([224, 224, 3]), params_learning_rate=0.01, params_include_top=False, params_weights='imagenet', params_classes=2)
